# Homework 8 - Deep Learning

Модифицируйте **TextCNN** с семинара и обучите ее на датасете IMDB. Добейтесь качества в 87% на тестовой выборке. Определим тестовую выборку следующим образом:

```python
df = pd.read_csv('IMDB Dataset.csv')
df_train, df_test = np.split(df, [45000], axis=0)
```

Модифицируйте класс датасета IMDB, чтобы он принимал датафрейм на вход, а не читал по пути.

Чтобы мы могли проверить задание, сдайте ваши предсказания для тестовой выборки (классы, считаем, что `positive = 0, negative = 1`). Для этого воспользуйтесь `torch.save`, сдайте тензор размером (5000).

## Загрузка библиотек

In [10]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import re
import os
import string
from typing import List, Tuple
from collections import Counter

## Импорт и определение датасета

In [22]:
df = pd.read_csv('../../IMDB Dataset.csv')
df_train, df_test = np.split(df, [45000], axis=0)

In [24]:
# Создаем класс словаря
class Vocabulary:
    def __init__(self, texts: List[str], min_freq: int = 10):
        text = ' '.join(texts)

        text = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", text)
        text = re.sub("[^a-zA-Z]", " ", text)

        while '  ' in text:
            text = text.replace('  ', ' ')

        words = text.strip().lower().split()

        c = Counter(words)

        self.vocabulary = list(set([word for word in words if c[word] >= min_freq]))
        self.vocabulary.append('<unk>')
        self._idx2word = {i: word for i, word in enumerate(self.vocabulary)}
        self._word2idx = {word: i for i, word in enumerate(self.vocabulary)}

    def get_vocabulary(self):
        return self.vocabulary

    def idx2word(self, idx: int):
        if idx not in self._idx2word:
            return '<unk>'

        return self._idx2word[idx]

    def word2idx(self, word: str):
        word = word.lower()
        if word not in self._word2idx:
            return self._word2idx['<unk>']

        return self._word2idx[word]
    
    def encode(self, text):
        result = []

        for word in text.split():
            result.append(self.word2idx(word))

        return result

    def build_vectors(self, fasttext):
        vectors = []
        
        for word in self.vocabulary:
            if fasttext.has_index_for(word):
                vectors.append(fasttext[word])
            else:
                vectors.append(np.zeros(25))

        return np.stack(vectors)

In [ ]:
class IMDB(Dataset):
    def __init__(self, path):
        self.df = pd.read_csv(path)
        
        texts = self.df['review'].values

        self.vocab = Vocabulary(texts, min_freq=5)

        self.label2idx = {'positive': 0, 'negative': 1}

    def __getitem__(self, idx):
        text = self.df['review'].loc[idx]
        label = self.label2idx[self.df['sentiment'].loc[idx]]

        text = torch.LongTensor(self.vocab.encode(text))
        label = torch.FloatTensor([label])

        return text, label

    def __len__(self):
        return len(self.df)

In [27]:
# функция для создания батчей и лоудер
from torch.nn.utils.rnn import pad_sequence

pad_idx = len(dataset.vocab.vocabulary)

def collate_fn(batch):
    texts = pad_sequence([b[0] for b in batch], padding_value=pad_idx, batch_first=True)
    labels = torch.stack([b[1] for b in batch])
    
    return texts, labels

train_loader = DataLoader(df_train, batch_size=64, collate_fn=collate_fn, pin_memory=True)
test_loader = DataLoader(df_test, batch_size=64, collate_fn=collate_fn, pin_memory=True)

AttributeError: 'DataFrame' object has no attribute 'vocab'